In [14]:
import praw
from prawcore import Forbidden
from pymongo import MongoClient, errors

In [2]:
creds = {}
with open ('/opt/cap1/.cap1', 'r') as fp:
    for line in fp:
        k, v = line.replace('\n','').split('\t')
        creds[k] = v

In [3]:
reddit = praw.Reddit(client_id=creds['REDDIT_ID'], 
    client_secret=creds['REDDIT_SECRET'],
    password=creds['REDDIT_PASSWORD'], 
    username=creds['REDDIT_USERNAME'],
    user_agent='accessAPI:v0.0.1 (by /u/{})'.format(creds['REDDIT_USERNAME']))

In [4]:
client = MongoClient('localhost', 27017)
db = client['cap2']

In [5]:
for coll in db.list_collection_names():
    print(f'{coll:14}: {db[coll].count_documents({}):6}')

submission    :  14523
first_and_last:    952
comment       :  69713


In [23]:
log = [] 
i = 0 
for submission_id in db['comment'].find().distinct('link_id')[0:1]: 
    i += 1 
    query_id = submission_id.split('_')[1] 
    print(f'searching for comments in {query_id}') 
    submission = reddit.submission(id=query_id) 
    try: 
        comments = submission.comments._comments  
    except Forbidden: 
        log.append(i) 
        continue 
    for comment in comments: 
        d = dict(vars(comment)) 
        for key in ['_replies', '_submission',  
                '_reddit', 'mod', 'author', 
                        'subreddit']: 
            _ = d.pop(key, None) 
        try: 
            db['comment'].insert_one(d) 
        except (errors.DuplicateKeyError, errors.InvalidDocument): 
            continue

searching for comments in 100ikq
